In [1]:
from src import geneticAlgo, follow_time
from src import plot_outcomes
import matplotlib.pyplot as plt
import tqdm
import pickle
import numpy as np
import pandas as pd

### Genetic algorithm

In [2]:
size_population = 200000
ratio_breeding = 0.7
size_tournament = 20000
gene_size = 30

algo = geneticAlgo(size_population = size_population, 
                   ratio_breeding = ratio_breeding, 
                   size_tournament = size_tournament,
                   gene_size = gene_size)

Outcome test symmetry: Validated.


In [3]:
D = {}
for option in algo.L_options:
    print(option.capitalize())
    D[option] = []
    for _ in tqdm.tqdm(range(10)):
        dt = []
        f0 = follow_time()
        algo.create_population()
        dt.append(f0.update())
        for i in range(20):
            algo.next_gen(option = option)
            dt.append(f0.update())
        ind = algo.indicators
        ind["time"] = dt

        D[option].append(ind)

Uniform


100%|██████████| 10/10 [01:11<00:00,  7.14s/it]


Elitist


100%|██████████| 10/10 [01:11<00:00,  7.18s/it]


Lottery


100%|██████████| 10/10 [01:28<00:00,  8.88s/it]


Tournament


100%|██████████| 10/10 [03:32<00:00, 21.20s/it]


Random


100%|██████████| 10/10 [01:45<00:00, 10.53s/it]


In [4]:
with open('Analysis/output.pickle', 'wb') as handle:
    pickle.dump(D, handle, protocol=pickle.HIGHEST_PROTOCOL)

### NEXT STEPS
- Implement TSP2
- Improve computing power using cuda (voir)

### Analysis

In [5]:
with open('Analysis/output.pickle', 'rb') as f:
    D = pickle.load(f)

In [6]:
# Figures indicators
for name_indicator in ['best_fitness', 'average_fitness', 'fitness_variance']:
    fig = plot_outcomes(D, name_indicator)
    plt.savefig(f"Analysis/{name_indicator}.png", dpi = 300); plt.close()

In [7]:
D_t = {}
for n, option in enumerate(D.keys()):
    out = np.zeros((21,10))

    for i, k in enumerate(D[option]):
        data = np.array(k["time"])
        out[:,i] = data
    D_opt = {"initialization (s.)":np.mean(out[0,:]), 
             "mean new gen. (s.)":np.mean(out[1:,:]), 
             "min new gen. (s.)":np.min(out[1:,:]), 
             "max new gen. (s.)":np.max(out[1:,:])}    
    D_t[option] = D_opt

In [8]:
df_time = pd.DataFrame.from_dict(D_t)
display(df_time)

,uniform,elitist,lottery,tournament,random
initialization (s.),0.590885,0.507398,0.502761,0.510071,0.502509
mean new gen. (s.),0.327331,0.333353,0.418885,1.034592,0.501101
min new gen. (s.),0.286640,0.324886,0.412559,1.006553,0.493928
max new gen. (s.),3.772464,1.552751,0.446900,2.801758,0.528330
